<a href="https://colab.research.google.com/github/srikarraju/GridWorld/blob/main/GridWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
class Gridworld:
  def __init__(self,grid_dim=4):
    self.grid_dim = grid_dim
    self.number_states = self.grid_dim*self.grid_dim - 1
    self.curr_state = 0

  def reset(self):
    self.curr_state = 0
    return self.curr_state

  def step(self,state,action):
    curr_state = (int(state/self.grid_dim),state%self.grid_dim)
    if action == 0:#Left
      if curr_state[1]-1<0:
        return state,-1,0
      else:
        next_state = (curr_state[0],curr_state[1]-1)
        if next_state[0]==self.grid_dim-1 and next_state[1]==self.grid_dim-1:
          return next_state[0]*4 + next_state[1],0,1
        else:
          return next_state[0]*4 + next_state[1],-1,0
    elif action ==1:#Right
      if curr_state[1]+1>=self.grid_dim:
        return state,-1,0
      else:
        next_state = (curr_state[0],curr_state[1]+1)
        if next_state[0]==self.grid_dim-1 and next_state[1]==self.grid_dim-1:
          return next_state[0]*4 + next_state[1],0,1
        else:
          return next_state[0]*4 + next_state[1],-1,0
    elif action ==2:#Up
      if curr_state[0]-1<0:
        return state,-1,0
      else:
        next_state = (curr_state[0]-1,curr_state[1])
        if next_state[0]==self.grid_dim-1 and next_state[1]==self.grid_dim-1:
          return next_state[0]*4 + next_state[1],0,1
        else:
          return next_state[0]*4 + next_state[1],-1,0
    elif action ==3:#Down
      if curr_state[0]+1>=self.grid_dim:
        return state,-1,0
      else:
        next_state = (curr_state[0]+1,curr_state[1])
        if next_state[0]==self.grid_dim-1 and next_state[1]==self.grid_dim-1:
          return next_state[0]*4 + next_state[1],0,1
        else:
          return next_state[0]*4 + next_state[1],-1,0
    else:
      print("Invalid Action")
      return None,0,1

In [ ]:
import numpy as np
from numpy.random import default_rng

rng = default_rng()

def generate_state_features(num_states,d,l):
  state_features = []
  product_dict = {}
  while len(state_features) < num_states:
    one_indices = rng.choice(d, size = l, replace=False)
    product = 1
    for i in range(l):
      product *= one_indices[i]
    feature_vec = np.zeros(d)
    if product not in product_dict:
      for index in one_indices:
        feature_vec[index] = 1
      state_features.append(feature_vec)
      product_dict[product] = 1
  #print(len(product_dict))
  return state_features

#print(np.asarray(generate_state_features(16,8,4)))

In [ ]:
def generate_state_action_features(state_vec, d, num_actions):
  state_action_features = np.zeros(shape=(num_actions,d*num_actions))
  for i in range(num_actions):
    for j in range(d):
      state_action_features[i][d*i+j] = state_vec[j]
  return state_action_features

print(generate_state_action_features([1,0],2,4))

# def generate_state_action_features(state_vec, d, num_actions):
#   state_action_features = []
#   for i in range(num_actions):
#     state_action_features.append(state_vec)
#     state_action_features[i] = np.append(state_action_features[i],i)
#   return np.asarray(state_action_features)

#print(generate_state_action_features(np.asarray([1,0]),2,4))

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
im
import torch.nn as nnport gym
from collections import deque
import numpy as np

In [ ]:
grid_dim = 4
num_actions = 4
d = 2

env = Gridworld(grid_dim)
#print(env)
returns = deque(maxlen=100)
weights_v = np.zeros(2)
weights_p = np.zeros(d*num_actions)
weights_w = np.zeros(d*num_actions)

print(weights_v)
print(weights_p)
print(weights_w)

[0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
alpha_0, beta_0, gamma, epsilon = 0.1, 0.01, 0.95, 0.1
alpha_c, beta_c = 1000, 100000
t = 0
n_episode = 0
actions_list = []
avg_reward = 0

state_features = np.array([[0,0],[0,1],[0,2],[0,3],
                           [1,0],[1,1],[1,2],[1,3],
                           [2,0],[2,1],[2,2],[2,3],
                           [3,0],[3,1],[3,2],[3,3]
                           ])
while n_episode <=5000:
  rewards,states,actions = [],[],[]
  state = 0
  t = 0
  episode_len = 0
  while episode_len<=100:
    episode_len += 1
    state_action_features = generate_state_action_features(state_features[state],d,num_actions)
    t += 1

    probs = np.dot(weights_p,np.transpose(state_action_features))

    probs -= probs.max()
    probs = np.exp(np.clip(probs/epsilon, -500, 500))
    probs /= probs.sum()
    probs2 = probs.cumsum()
    action = np.where(probs2 >= np.random.random())[0][0]

    probs3 = -1*probs
    probs3[action] += 1
    features = state_features[state]
    features = np.repeat(features.reshape((features.size,1)), num_actions, axis=1)
    compat_features = np.dot(features, np.diag(probs3))



    new_state, reward, done = env.step(state,action)


    value_curr = np.dot(weights_v,np.asarray(state_features[state]))
    value_next = np.dot(weights_v,np.asarray(state_features[new_state]))

    avg_reward = (1 - gamma)*avg_reward + gamma * reward

    #td_error = reward + value_curr - value_next - avg_reward

    td_error = reward + value_curr - value_next

    beta = (beta_0 * beta_c) / (beta_c + t)
    alpha = (alpha_0 * alpha_c) / (alpha_c + t**(2/3))

    weights_v += alpha * (abs(td_error)/td_error) * np.asarray(state_features[state])

    grad_prob = (1 - probs[action]) * np.asarray(state_action_features[action])

    weights_w = np.dot(np.eye(compat_features.size) - beta * np.outer(compat_features, compat_features), weights_w)
    weights_p += beta * weights_w


    states.append(state)
    actions.append(action)
    rewards.append(reward)
    actions_list.append(action)

    state = new_state
    if done == 1:
      break

  returns.append(np.sum(rewards))
  #print(np.sum(rewards))
  if n_episode%100==0:
    print("Episode: {:6d}\tAvg. Return: {:6.2f}".format(n_episode, np.mean(returns)))
  n_episode += 1

Episode:      0	Avg. Return: -37.00
Episode:    100	Avg. Return: -52.99
Episode:    200	Avg. Return: -52.93
Episode:    300	Avg. Return: -49.56
Episode:    400	Avg. Return: -53.93
Episode:    500	Avg. Return: -48.31
Episode:    600	Avg. Return: -55.17
Episode:    700	Avg. Return: -51.36
Episode:    800	Avg. Return: -50.70
Episode:    900	Avg. Return: -47.63
Episode:   1000	Avg. Return: -46.54
Episode:   1100	Avg. Return: -55.28
Episode:   1200	Avg. Return: -52.02
Episode:   1300	Avg. Return: -45.62
Episode:   1400	Avg. Return: -53.11
Episode:   1500	Avg. Return: -52.50
Episode:   1600	Avg. Return: -49.16
Episode:   1700	Avg. Return: -55.78
Episode:   1800	Avg. Return: -50.07
Episode:   1900	Avg. Return: -57.08
Episode:   2000	Avg. Return: -50.88
Episode:   2100	Avg. Return: -50.67
Episode:   2200	Avg. Return: -52.33
Episode:   2300	Avg. Return: -52.47
Episode:   2400	Avg. Return: -49.92
Episode:   2500	Avg. Return: -47.24
Episode:   2600	Avg. Return: -53.02
Episode:   2700	Avg. Return:

In [ ]:
alpha_0, beta_0, gamma, epsilon = 0.1, 0.01, 0.95, 0.1
alpha_c, beta_c = 1000, 100000
t = 0
n_episode = 0
actions_list = []
avg_reward = 0

state_features = np.array([[0,0],[0,1],[0,2],[0,3],
                           [1,0],[1,1],[1,2],[1,3],
                           [2,0],[2,1],[2,2],[2,3],
                           [3,0],[3,1],[3,2],[3,3]
                           ])
while n_episode <=500000:
  rewards,states,actions = [],[],[]
  state = env.reset()

  state = 0
  value_curr = np.dot(state_features[state],weights_v)
  episode_len = 0
  #t = 0
  #print(n_episode)
  #print(value_curr)
  while True:
    t += 1
    episode_len += 1
    state_action_features = generate_state_action_features(state_features[state],d,num_actions)
    probs = np.dot(np.asarray(state_action_features),weights_p)
    probs -= probs.max()
    probs = np.exp(np.clip(probs/epsilon, -50, 50))
    probs /= probs.sum()

    probs2 = probs.cumsum()
    action = np.where(probs2 >= np.random.random())[0][0]
    #print(action)

    new_state, reward, done = env.step(state,action)


    value_curr = np.dot(weights_v,np.asarray(state_features[state]))
    value_next = np.dot(weights_v,np.asarray(state_features[new_state]))

    beta = (beta_0 * beta_c) / (beta_c + t)
    alpha = (alpha_0 * alpha_c) / (alpha_c + t**(2/3))

    avg_reward = (1 - gamma*alpha)*avg_reward + gamma*alpha * reward

    td_error = reward + value_next - value_curr - avg_reward

    weights_v += 0.01*alpha * td_error * np.asarray(state_features[state])

    grad_prob = probs[action]*(state_action_features[action] - np.dot(np.transpose(state_action_features),probs)) #change this as probs is cum sum

    weights_w = np.dot((np.eye(d+1) - alpha * np.outer(grad_prob,grad_prob) ),weights_w) + alpha * td_error * grad_prob

    weights_p += beta * weights_w

    states.append(state)
    actions.append(action)
    rewards.append(reward)
    actions_list.append(action)

    state = new_state
    if done==1 or episode_len>=200:
      break
  n_episode += 1
  returns.append(np.sum(rewards))
  #print(np.sum(rewards))
  if n_episode%100==0:
    print("Episode: {:6d}\tAvg. Return: {:6.2f}".format(n_episode, np.mean(returns)))

NameError: ignored

In [ ]:
print(np.dot(state_features,weights_v))

[ 0.         -0.17910808 -0.35821617 -0.53732425  1.15368516  0.97457707
  0.79546899  0.61636091  2.30737031  2.12826223  1.94915414  1.77004606
  3.46105547  3.28194738  3.1028393   2.92373122]


In [ ]:
print(states)

[0, 4, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12]


In [ ]:
state = 12
probs = np.dot(generate_state_action_features(state_features[state],d,num_actions),weights_p)
probs -= probs.max()

probs = np.exp(np.clip(probs/epsilon, -50, 50))
probs /= probs.sum()
print(probs)

[1.92874985e-22 1.92874985e-22 1.92874985e-22 1.00000000e+00]


In [ ]:
env = Gridworld(4)
state = env.reset()
total_reward = 0.0
for _ in range(10000):
  #print(state)
  action = np.random.randint(0,4)
  #print(action)
  next_state, reward, done  = env.step(state,action)#take a random action
  total_reward += reward
  state = next_state
  if done== 1:
    #print(action)
    print(total_reward)
    state = env.reset()
    total_reward = 0